<a href="https://colab.research.google.com/github/volynsal/Earnings-Call-Summarization/blob/main/Earnings_Call_Summarization_(v2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


Once openai package is installed, we need to import it.

In [ ]:
import openai

Now that we have an API key, we need to provide it to the OpenAI Python package. Since I want to input my API key without showing it here, I am going to import getpass and prompt myself for the API Key:

In [ ]:
from getpass import getpass
openai.api_key = getpass()

··········


First, let's read the earnings call text into a variable. We can use the requests library to fetch the raw text of the Nvidia earnings call:

In [ ]:
import requests

url = "https://www.fool.com/earnings/call-transcripts/2023/11/22/nvidia-nvda-q3-2024-earnings-call-transcript/"
response = requests.get(url)
transcript = response.text

transcript

'\n\n<!DOCTYPE html>\n<html lang="en" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article#">\n<head>\n\n\n<script type="text/javascript" src="https://cdn.cookielaw.org/consent/02abb198-81a8-49e5-a9b1-f69a5dd9c039/OtAutoBlock.js"></script>\n<script src="https://cdn.cookielaw.org/scripttemplates/otSDKStub.js" type="text/javascript" charset="UTF-8" data-domain-script="02abb198-81a8-49e5-a9b1-f69a5dd9c039"></script>\n\n<script>\nfunction OptanonWrapper(){var t;window.infotrackInitialized||void 0===window.Infotrack?window.Infotrack.setSegmentIntegrations():(window.infotrackInitialized=!0,Infotrack.load("https://g.foolcdn.com/mms/resources/js/infotrack-ot_min.js"),t=t||0,setTimeout(function(){window.Infotrack.initialize("usmf")},t))}window.top!=window&&(window.analytics=window.top.analytics,window.inIframe=!0);var segmentKey="CL257pIDks5ukrbYAs7Ez3fkLz1bWVnJ",segmentSnippetVersion="4.15.2",getSegmentUrl=function(t){return t=t||window.segmentKey,("https:"=

Now that we have the earnings text in a data variable, let's make this into a summarization prompt. To do this, we can simply append "tldr;" and some carriage returns to the data:

In [ ]:
prompt = f"{transcript}\n\ntl;dr:"

prompt

'\n\n<!DOCTYPE html>\n<html lang="en" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article#">\n<head>\n\n\n<script type="text/javascript" src="https://cdn.cookielaw.org/consent/02abb198-81a8-49e5-a9b1-f69a5dd9c039/OtAutoBlock.js"></script>\n<script src="https://cdn.cookielaw.org/scripttemplates/otSDKStub.js" type="text/javascript" charset="UTF-8" data-domain-script="02abb198-81a8-49e5-a9b1-f69a5dd9c039"></script>\n\n<script>\nfunction OptanonWrapper(){var t;window.infotrackInitialized||void 0===window.Infotrack?window.Infotrack.setSegmentIntegrations():(window.infotrackInitialized=!0,Infotrack.load("https://g.foolcdn.com/mms/resources/js/infotrack-ot_min.js"),t=t||0,setTimeout(function(){window.Infotrack.initialize("usmf")},t))}window.top!=window&&(window.analytics=window.top.analytics,window.inIframe=!0);var segmentKey="CL257pIDks5ukrbYAs7Ez3fkLz1bWVnJ",segmentSnippetVersion="4.15.2",getSegmentUrl=function(t){return t=t||window.segmentKey,("https:"=

Now that we have a prompt, let's call OpenAI using this prompt:

In [ ]:
# Replace 'your_openai_api_key' with your actual OpenAI API key
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + openai.api_key,
}

data = {
    "model": "gpt-4-0125-preview",
    "messages": [{"role": "user", "content": prompt}],
    "temperature": 0.7,
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=data)

print(response.json()['choices'][0]['message']['content'])


Notice that the summary, while adequate, may not provide enough detail for power users. 'Chunking' allows us to feed pieces of the earnings call into the OpenAI API, allowing our model to focus on each piece at a time. This results in a much more detailed, intelligent summary that does not repeat facts as it now has context from the prior messages. This will take more time to load but the reward may be worth it for some. You will see this below shortly!

There are many ways to slice and dice text. Let's try first splitting up our text into a list of words, then dividing that list of words into equal chunks using numpy.

In [ ]:
import numpy as np

words = transcript.split(" ")
chunks = np.array_split(words, 6)

chunks

[array(['NVDA', 'earnings', 'call', ..., 'NVIDIA', 'GPUs.', 'We'],
       dtype='<U33'),
 array(['also', 'announced', 'the', ..., 'grow?', 'Just', 'where'],
       dtype='<U33'),
 array(['are', 'we', 'in', ..., 'Bluefield', 'as', 'well'], dtype='<U33'),
 array(['as', 'our', 'Spectrum', ..., 'Thank', 'you.', 'On'], dtype='<U33'),
 array(['the', 'question', 'regarding', ..., 'and', 'as', 'a'],
       dtype='<U33'),
 array(['software', 'application', 'provider,', ..., 'Company', '--',
        'Analyst'], dtype='<U33')]

Let's now loop through all of the chunks and save the responses in a list called summary_responses. At the end we'll join all of the responses into a single summary and check out the output.

In [ ]:
summary_responses = []

for chunk in chunks:

    sentences = ' '.join(list(chunk))

    prompt = f"{sentences}\n\ntl;dr:"

    data = {
        "model": "gpt-4-0125-preview",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7,
    }

    response_text = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=data).json()
    cleansed_response = response_text['choices'][0]['message']['content']
    summary_responses.append(cleansed_response)

full_summary = "".join(summary_responses)

print("Full Summary\n")
print(full_summary)

Full Summary

NVIDIA announced a record-breaking third quarter for its fiscal year 2024, with revenue reaching $18.1 million, marking a 34% increase sequentially and more than a 200% increase year-over-year, significantly surpassing the anticipated $16 billion. The surge was primarily driven by its data center business, which saw record revenue of $14.5 billion due to the NVIDIA HGX platform based on the Hopper Tensor Core GPU architecture, along with InfiniBand networking. The company highlighted strong demand for its AI computing solutions across consumer internet companies and enterprises, as well as continued investment in infrastructure for AI applications. NVIDIA also addressed the impact of new U.S. export control regulations but expects strong growth in other regions to offset potential declines in affected markets. The call also touched on NVIDIA's efforts to expand its data center product portfolio to comply with these regulations and its work with countries on building sover